<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Assignment 7: Output Analysis (Multiple Model)

Summer Semester 24


Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

* Output Analysis (Multiple Model)
* Metamodels


In [1]:
import simpy
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

## Car wash example

In [2]:
class EventLogger:
    def __init__(self):
        self.logs = []
        self.replication = None
        self.seed = None
        self.system_id = None #used to identify the system in the logs
    
    def set_system_id(self, system_id):
        self.system_id = system_id

    def set_replication_info(self, replication, seed):
        self.replication = replication
        self.seed = seed

    def log_car_arrival(self, entity, time, interarrival_time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_arrival', 'event_key': entity, #simulation info
                          'interarrival_time': interarrival_time
                          })
    
    def log_car_wash_request(self, entity, time, machine_id, processing_time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_wash_request', 'event_key': entity, #simulation info
                          'machine_id': machine_id, 'processing_time': processing_time
                          })

    def log_car_departure(self, entity, time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_departure', 'event_key': entity #simulation info
                          })

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

#### System 1

2 Machines with a common queue

In [3]:
class Carwash_S1:
    def __init__(self, env, logger, n_machines=2):
        self.env = env
        self.machine = simpy.Resource(env, n_machines)
        self.logger = logger

    def wash(self, processing_time):
        yield self.env.timeout(processing_time)

In [4]:
class Car_S1:
    def __init__(self, env, name, carwash, logger, interarrival_time, processing_time):
        self.env = env
        self.name = name
        self.logger = logger
        self.interarrival_time = interarrival_time
        self.processing_time = processing_time

        self.env.process(self.run(carwash))

    def run(self, carwash):
        
        # Log the arrival
        self.logger.log_car_arrival(self.name, self.env.now, self.interarrival_time)
        with carwash.machine.request() as request:
            yield request

            # Log the request for car wash
            self.logger.log_car_wash_request(self.name, self.env.now, 'unique_machine', self.processing_time)
            yield self.env.process(carwash.wash(self.processing_time))

            #Log the departure
            self.logger.log_car_departure(self.name, self.env.now)

#### System 2

- 2 Machines with a dedicated queue
- Entities pick the machine with shorter queue

In [5]:
class Carwash_S2:
    def __init__(self, env, logger):
        self.env = env
        self.machine1 = simpy.Resource(env, 1)
        self.machine2 = simpy.Resource(env, 1)
        self.logger = logger

    def wash(self, processing_time):
        yield self.env.timeout(processing_time)

In [6]:
class Car_S2:
    def __init__(self, env, name, carwash, logger, interarrival_time, processing_time):
        self.env = env
        self.name = name
        self.logger = logger
        self.interarrival_time = interarrival_time
        self.processing_time = processing_time

        self.env.process(self.run(carwash))

    def run(self, carwash):
        
        # Log the arrival
        self.logger.log_car_arrival(self.name, self.env.now, self.interarrival_time)

        # Check which machine has the shortest queue
        if len(carwash.machine1.queue) <= len(carwash.machine2.queue):
            with carwash.machine1.request() as request: 
                yield request

                # Log the request for car wash
                self.logger.log_car_wash_request(self.name, self.env.now, 'machine_1', self.processing_time)
                yield self.env.process(carwash.wash(self.processing_time))
        else:
            with carwash.machine2.request() as request:
                yield request

                # Log the request for car wash
                self.logger.log_car_wash_request(self.name, self.env.now, 'machine_2', self.processing_time)
                yield self.env.process(carwash.wash(self.processing_time))

        #Log the departure
        self.logger.log_car_departure(self.name, self.env.now)            

#### Entity generator

In [7]:
def car_generator(env, system, carwash, logger):
    '''
    system: system configuration
    '''
    car_count = 0
    
    # Create cars while the simulation is running
    while True:
        iat = np.random.exponential(1/0.5) #interarrival time
        pt = np.random.exponential(1/0.3) #processing time

        yield env.timeout(iat)

        if system == 1:
            Car_S1(env, f'Car {car_count}', carwash, logger, iat, pt)
        elif system == 2:
            Car_S2(env, f'Car {car_count}', carwash, logger, iat, pt)
        car_count += 1

## Run Simulation

In [37]:
# parameters
SIM_TIME = 8*60    # Simulation time in minutes
N_REPLICATIONS = 10 # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger = EventLogger()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)

    #set replication id and random seed
    logger.set_replication_info(i, seed)

    ######################################
    ############## SYSTEM 1 ##############
    ######################################

    #set system id
    logger.set_system_id(1)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s1 = Carwash_S1(env, logger)

    #define processes
    env.process(car_generator(env, 1, carwash_s1, logger))

    # Execute
    env.run(until=SIM_TIME)

    ######################################
    ############## SYSTEM 2 ##############
    ######################################
    
    #set system id
    logger.set_system_id(2)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s2 = Carwash_S2(env, logger)

    #define processes
    env.process(car_generator(env, 2, carwash_s2, logger))

    # Execute
    env.run(until=SIM_TIME)

print('... Done')

Running Simulation...
Running Replication 0 with seed: 91 ...
Running Replication 1 with seed: 90 ...
Running Replication 2 with seed: 27 ...
Running Replication 3 with seed: 59 ...
Running Replication 4 with seed: 61 ...
Running Replication 5 with seed: 70 ...
Running Replication 6 with seed: 99 ...
Running Replication 7 with seed: 29 ...
Running Replication 8 with seed: 31 ...
Running Replication 9 with seed: 86 ...
... Done


In [38]:
events_df = logger.get_logs_df()

events_df

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,91,0.448798,car_arrival,Car 0,0.448798,NaN,NaN
1,1,0,91,0.448798,car_wash_request,Car 0,NaN,unique_machine,1.330055
2,1,0,91,1.152172,car_arrival,Car 1,0.703374,NaN,NaN
3,1,0,91,1.152172,car_wash_request,Car 1,NaN,unique_machine,0.326603
4,1,0,91,1.478774,car_departure,Car 1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14244,2,9,86,478.169763,car_arrival,Car 262,2.940086,NaN,NaN
14245,2,9,86,478.576807,car_departure,Car 239,NaN,NaN,NaN
14246,2,9,86,478.576807,car_wash_request,Car 242,NaN,machine_1,1.358596
14247,2,9,86,479.935403,car_departure,Car 242,NaN,NaN,NaN


In [39]:
boolean_mask = (events_df['replication_id'] == 0) & (events_df['event_key'].isin(['Car 0', 'Car 1']) )

events_df[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,91,0.448798,car_arrival,Car 0,0.448798,NaN,NaN
610,2,0,91,3.642769,car_arrival,Car 0,3.642769,NaN,NaN
5,1,0,91,1.778853,car_departure,Car 0,NaN,NaN,NaN
612,2,0,91,4.032888,car_departure,Car 0,NaN,NaN,NaN
1,1,0,91,0.448798,car_wash_request,Car 0,NaN,unique_machine,1.330055
611,2,0,91,3.642769,car_wash_request,Car 0,NaN,machine_1,0.390119
2,1,0,91,1.152172,car_arrival,Car 1,0.703374,NaN,NaN
613,2,0,91,5.786456,car_arrival,Car 1,2.143687,NaN,NaN
4,1,0,91,1.478774,car_departure,Car 1,NaN,NaN,NaN
616,2,0,91,6.069056,car_departure,Car 1,NaN,NaN,NaN


In [40]:
boolean_mask = (events_df['replication_id'] == 1) & (events_df['event_key'].isin(['Car 0', 'Car 1']) )

events_df[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
1384,1,1,90,0.332237,car_arrival,Car 0,0.332237,NaN,NaN
2132,2,1,90,4.818321,car_arrival,Car 0,4.818321,NaN,NaN
1386,1,1,90,0.904244,car_departure,Car 0,NaN,NaN,NaN
2134,2,1,90,5.349655,car_departure,Car 0,NaN,NaN,NaN
1385,1,1,90,0.332237,car_wash_request,Car 0,NaN,unique_machine,0.572007
2133,2,1,90,4.818321,car_wash_request,Car 0,NaN,machine_1,0.531335
1387,1,1,90,8.798225,car_arrival,Car 1,8.465988,NaN,NaN
2135,2,1,90,5.675111,car_arrival,Car 1,0.856791,NaN,NaN
1400,1,1,90,16.626909,car_departure,Car 1,NaN,NaN,NaN
2144,2,1,90,8.088179,car_departure,Car 1,NaN,NaN,NaN


# Exercise / Tasks

**Tasks are independently of each other.**

## Task 1: Comparison of Two System Designs (CRN)

We'd like to compare the **average waiting time** of System 1 and System 2 but with Common Random Numbers (CRN). There are several ways of implementing this but today we'll guide you with one of them.

When implementing CRN, is important to note the following:

- It is never enough to simply use the same seed on the random-number generator(s). 
- Each random number used in one model for some purpose should be used for the same purpose in the second model—that is, the use of the random numbers must be synchronized. 
- For example, if the $i$ random number is used to generate the processing time of a machine for the 5th car in model 1, then the $i$ random number should be used for the very same purpose in model 2. 

-----

**Task 1.1: Create a stream of random numbers for the interarrival times and processing times. Use the same distributions as before, i.e. $exp(1/0.5)$ and $exp(1/0.3)$, respectively. The size of the stream should be at least 1000. Note that this stream must change whenever we set a new seed but must remain the same across system designs**

**Task 1.2: Modify the `car_generator_crn` to receive this stream of numbers (both, interarrival and processing times) and use them according the counter for each car. Modify also the parts of your code where you're calling this function.**

After these first 2 steps. You should have succesfully implemented CRN.

**Task 1.3: Create the "replication table". That is: a table that contains for every replication (<u>and now, for every system design</u>) the average waiting time within replications. This should be very similar to Task 1 of Assignment 6 but considering that now we have two different system designs.**

**Task 1.4: Evaluate whether the difference of average waiting times between the two system designs is statistically significant. You can use a CI or a hypothesis test for this purpose. Recall that when using CRN, we calculate the “synchronized” performance difference for each replication and then we conduct the CI or hypothesis test. Check slide 23 of the Lecture "Output Analysis of Multiple Models" for more details.**

-----

In [12]:
def car_generator_crn(env, system, carwash, logger, stream_iat, stream_pt):
    '''
    system: system configuration
    stream_iat: inter-arrival time stream
    stream_pt: processing time stream
    '''
    car_count = 0
    
    # Create cars while the simulation is running
    while True:
        iat = stream_iat[car_count] #inter-arrival time for car "i"
        pt = stream_pt[car_count] #processing time time for car "i"

        yield env.timeout(iat) #common random numbers

        if system == 1:
            Car_S1(env, f'Car {car_count}', carwash, logger, iat, pt)
        elif system == 2:
            Car_S2(env, f'Car {car_count}', carwash, logger, iat, pt)
        car_count += 1

In [13]:
# parameters
SIM_TIME = 8*60    # Simulation time in minutes
N_REPLICATIONS = 10 # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger_t1 = EventLogger()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)

    #generate streams of inter-arrival times and processing times (common random numbers)
    stream_iat = np.random.exponential(1/0.5, 10000)
    stream_pt = np.random.exponential(1/0.3, 10000)

    #set replication id and random seed
    logger_t1.set_replication_info(i, seed)

    ######################################
    ############## SYSTEM 1 ##############
    ######################################

    #set system id
    logger_t1.set_system_id(1)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s1 = Carwash_S1(env, logger_t1)

    #define processes
    env.process(car_generator_crn(env, 1, carwash_s1, logger_t1, stream_iat, stream_pt))

    # Execute
    env.run(until=SIM_TIME)

    ######################################
    ############## SYSTEM 2 ##############
    ######################################

    #set system id
    logger_t1.set_system_id(2)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s2 = Carwash_S2(env, logger_t1)

    #define processes
    env.process(car_generator_crn(env, 2, carwash_s2, logger_t1, stream_iat, stream_pt))

    # Execute
    env.run(until=SIM_TIME)

print('... Done')

Running Simulation...
Running Replication 0 with seed: 48 ...
Running Replication 1 with seed: 61 ...
Running Replication 2 with seed: 21 ...
Running Replication 3 with seed: 43 ...
Running Replication 4 with seed: 62 ...
Running Replication 5 with seed: 47 ...
Running Replication 6 with seed: 87 ...
Running Replication 7 with seed: 16 ...
Running Replication 8 with seed: 69 ...
Running Replication 9 with seed: 22 ...
... Done


In [14]:
#Leave this code unchanged
events_df_t1 = logger_t1.get_logs_df()

events_df_t1

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,48,0.035290,car_arrival,Car 0,0.035290,NaN,NaN
1,1,0,48,0.035290,car_wash_request,Car 0,NaN,unique_machine,3.097991
2,1,0,48,3.133281,car_departure,Car 0,NaN,NaN,NaN
3,1,0,48,4.478651,car_arrival,Car 1,4.443361,NaN,NaN
4,1,0,48,4.478651,car_wash_request,Car 1,NaN,unique_machine,2.170864
...,...,...,...,...,...,...,...,...,...
14234,2,9,22,476.619557,car_departure,Car 231,NaN,NaN,NaN
14235,2,9,22,476.619557,car_wash_request,Car 232,NaN,machine_2,0.595313
14236,2,9,22,477.214869,car_departure,Car 232,NaN,NaN,NaN
14237,2,9,22,479.534262,car_arrival,Car 233,3.896078,NaN,NaN


In [15]:
#Leave this code unchanged (check that the interarrival times and processing times are the same for the same car in both systems)
boolean_mask = (events_df_t1['replication_id'] == 0) & (events_df_t1['event_key'].isin(['Car 0', 'Car 1']) )

events_df_t1[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,48,0.035290,car_arrival,Car 0,0.035290,NaN,NaN
700,2,0,48,0.035290,car_arrival,Car 0,0.035290,NaN,NaN
2,1,0,48,3.133281,car_departure,Car 0,NaN,NaN,NaN
702,2,0,48,3.133281,car_departure,Car 0,NaN,NaN,NaN
1,1,0,48,0.035290,car_wash_request,Car 0,NaN,unique_machine,3.097991
701,2,0,48,0.035290,car_wash_request,Car 0,NaN,machine_1,3.097991
3,1,0,48,4.478651,car_arrival,Car 1,4.443361,NaN,NaN
703,2,0,48,4.478651,car_arrival,Car 1,4.443361,NaN,NaN
8,1,0,48,6.649516,car_departure,Car 1,NaN,NaN,NaN
708,2,0,48,6.649516,car_departure,Car 1,NaN,NaN,NaN


In [16]:
#Leave this code unchanged (check that the interarrival times and processing times are the same for the same car in both systems)
boolean_mask = (events_df_t1['replication_id'] == 1) & (events_df_t1['event_key'].isin(['Car 0', 'Car 1']) )

events_df_t1[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
1398,1,1,61,3.479885,car_arrival,Car 0,3.479885,NaN,NaN
2132,2,1,61,3.479885,car_arrival,Car 0,3.479885,NaN,NaN
1400,1,1,61,3.710280,car_departure,Car 0,NaN,NaN,NaN
2134,2,1,61,3.710280,car_departure,Car 0,NaN,NaN,NaN
1399,1,1,61,3.479885,car_wash_request,Car 0,NaN,unique_machine,0.230396
2133,2,1,61,3.479885,car_wash_request,Car 0,NaN,machine_1,0.230396
1401,1,1,61,3.880081,car_arrival,Car 1,0.400197,NaN,NaN
2135,2,1,61,3.880081,car_arrival,Car 1,0.400197,NaN,NaN
1405,1,1,61,8.622953,car_departure,Car 1,NaN,NaN,NaN
2138,2,1,61,8.622953,car_departure,Car 1,NaN,NaN,NaN


In [17]:
#Compute the waiting time for each car in each system and each replication (you may want to use pd.pivot_table for this)
results_df_t1 = pd.pivot_table(events_df_t1, 
                            values='event_time', 
                            index=['system_id', 'replication_id', 'seed', 'event_key'], 
                            columns=['event_name'], aggfunc="sum")\
                .reset_index(drop=False)

#compute waiting time
results_df_t1['waiting_time'] = results_df_t1['car_wash_request'] - results_df_t1['car_arrival']

print(results_df_t1.columns)

results_df_t1

Index(['system_id', 'replication_id', 'seed', 'event_key', 'car_arrival',
       'car_departure', 'car_wash_request', 'waiting_time'],
      dtype='object', name='event_name')


event_name,system_id,replication_id,seed,event_key,car_arrival,car_departure,car_wash_request,waiting_time
0,1,0,48,Car 0,0.035290,3.133281,0.035290,0.000000
1,1,0,48,Car 1,4.478651,6.649516,4.478651,0.000000
2,1,0,48,Car 10,17.683431,28.342616,22.380644,4.697212
3,1,0,48,Car 100,183.085451,184.204259,183.085451,0.000000
4,1,0,48,Car 101,184.085138,185.857379,184.204259,0.119121
...,...,...,...,...,...,...,...,...
4785,2,9,22,Car 95,193.389097,195.338632,193.389097,0.000000
4786,2,9,22,Car 96,196.646885,197.563960,196.646885,0.000000
4787,2,9,22,Car 97,199.943992,200.317426,199.943992,0.000000
4788,2,9,22,Car 98,200.063392,200.785358,200.317426,0.254035


In [18]:
results_df_t1\
    .loc[results_df_t1['replication_id'] == 0]\
    .sort_values(by='car_arrival', ascending=True)\
    .head(10)

event_name,system_id,replication_id,seed,event_key,car_arrival,car_departure,car_wash_request,waiting_time
0,1,0,48,Car 0,0.035290,3.133281,0.035290,0.000000
2395,2,0,48,Car 0,0.035290,3.133281,0.035290,0.000000
1,1,0,48,Car 1,4.478651,6.649516,4.478651,0.000000
2396,2,0,48,Car 1,4.478651,6.649516,4.478651,0.000000
112,1,0,48,Car 2,5.149208,6.676326,5.149208,0.000000
2507,2,0,48,Car 2,5.149208,8.176633,6.649516,1.500307
161,1,0,48,Car 3,5.859636,10.065736,6.649516,0.789880
2556,2,0,48,Car 3,5.859636,9.275856,5.859636,0.000000
172,1,0,48,Car 4,9.000400,11.550121,9.000400,0.000000
2567,2,0,48,Car 4,9.000400,11.550121,9.000400,0.000000


In [19]:
#Compute the average waiting time for each system and each replication (average waiting time within replications, for the two systems)
within_replication_stats_df_t1 = results_df_t1\
    .groupby(['system_id', 'replication_id'])\
    .agg({'waiting_time': ['mean']})\
    .reset_index(drop=False)

print(within_replication_stats_df_t1.columns)
within_replication_stats_df_t1.head()

MultiIndex([(     'system_id',     ''),
            ('replication_id',     ''),
            (  'waiting_time', 'mean')],
           names=['event_name', None])


event_name,system_id,replication_id,waiting_time
,,,mean
0,1,0,3.233210
1,1,1,4.082083
2,1,2,4.480178
3,1,3,5.457967
4,1,4,4.139026


In [20]:
within_replication_stats_df_t1 = pd.pivot_table(within_replication_stats_df_t1, 
                                             values='waiting_time', 
                                             index=['replication_id'], 
                                             columns=['system_id'], aggfunc="mean")\
                                .reset_index(drop=False)

print(within_replication_stats_df_t1.columns)
within_replication_stats_df_t1.head()

MultiIndex([('replication_id', ''),
            (          'mean',  1),
            (          'mean',  2)],
           names=[None, 'system_id'])


/var/folders/td/mhppvrk10nj_1j1gl_dk2pvc0000gn/T/ipykernel_85087/3997147012.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  within_replication_stats_df_t1 = pd.pivot_table(within_replication_stats_df_t1,


replication_id      mean          
system_id                        1         2
0                      0  3.233210  4.311200
1                      1  4.082083  5.328726
2                      2  4.480178  6.077399
3                      3  5.457967  6.565718
4                      4  4.139026  5.442606

In [21]:
#Evaluate whether the difference of average waiting times between the two system designs is statistically significant

#significance level
alpha = 0.05

#compute mean, std and sample size
sample_size = N_REPLICATIONS
D = within_replication_stats_df_t1[('mean', 1)] - within_replication_stats_df_t1[('mean', 2)]
sample_mean = np.mean(D)
sample_std = np.std(D, )

print(f"sample mean: {sample_mean}")
print(f"sample std: {sample_std}")

#Calculate t confidence interval

# Calculate the t-value for a 95% confidence interval (two-sided)
t_value = stats.t.ppf(1-alpha/2, sample_size - 1) 

# Calculate the half-width of the confidence interval
ci_halfwidth = t_value * sample_std / np.sqrt(sample_size)

#calculate the lower and upper bounds of the confidence interval
ci_lb = sample_mean - ci_halfwidth
ci_ub = sample_mean + ci_halfwidth

print(f"t confidence int: [{ci_lb}, {ci_ub}]") 

sample mean: -1.1679087249756726
sample std: 0.2720438650035921
t confidence int: [-1.3625171825280216, -0.9733002674233237]


Answer the following questions:

- Which system configuration is better? Is the difference in performance statistically significant?

**Answer in this markdown chunk:**

- *Your answer here* (**Answer:** System 1 (2 Machines with a common queue) has a lower average waiting time than system 2 (2 Machines with a dedicated queue). The result is statistically significant (because the CI is totally to the left of 0))

## Task 2: Metamodels (CRN)

We'd like to create a metamodel for analyzing the **average waiting time** of System 1. Suppose that for reducing the waiting time, we can:
1. Buy more machines (i.e. increase the number of "counters")
1. Upgrade the current machines and therefore decrease the processing times by a so-called reduction factor. That is, if the reduction factor ($rf$) is 10% that means that we're decreasing the processing times by 10%. The formula is: $pt_{upgrade} = pt * (1 - rf)$
1. A combination of both options.

The idea is to understand how our output metric varies when varying this two design variables, we'll use a simple metamodel to approximate the relationship between our design variables and the average waiting time. Recall that we can use Independent Sampling or Common Random Numbers (CRN) when working with metamodels. In this occasion, we'll implement CRN.

When implementing CRN, is important to note the following:

- It is never enough to simply use the same seed on the random-number generator(s). 
- Each random number used in one model for some purpose should be used for the same purpose in the second model—that is, the use of the random numbers must be synchronized. 
- For example, if the $i$ random number is used to generate the processing time of a machine for the 5th car in model 1, then the $i$ random number should be used for the very same purpose in model 2. 

-----

(If you did Task 1, Task 2.1 and 2.2 are exactly the same as before)

**Task 2.1: Create a stream of random numbers for the interarrival times and processing times. Use the same distributions as before, i.e. $exp(1/0.5)$ and $exp(1/0.3)$, respectively. The size of the stream should be at least 1000. Note that this stream must change whenever we set a new seed but must remain the same across the design variables**

**Task 2.2: Modify the `car_generator_crn` to receive this stream of numbers (both, interarrival and processing times) and use them according the counter for each car. Modify also the parts of your code where you're calling this function.**

After these first 2 steps. You should have succesfully implemented CRN.

**Task 2.3: Within the *for* loop for iterate over the replications, nest two more *for* loops for iterate over the number of machines and the reduction factor. In particular, try with a number of machines from 1 to 4 (all integers) and a reduction factor from 0 (no reduction) to 80% with a stepsize of 10%.**

Note that for every seed, for every value of number of machines and for every value of reduction factor, you should run your simulation model. Don't forget to adjust the stream of processing times by the reduction factor and to set the corresponding number of machines when defining the Carwash object.

**Task 2.4: Modify the `EventLogger_Task2` to set the value of the current number of machines and reduction factor (you can implement this with a new method similar to `set_replication_info`). Include this information everytime you log an event (i.e. in every method that starts with `log_`)**

**Task 2.5: Create the "replication table". That is: a table that contains for every replication (<u>and now, for every combination of the design variables</u>) the average waiting time within replications. This should be very similar to Task 1 of Assignment 6 but considering that now we have two design variables.**

**Task 2.6: Aggregate the "replication table" accross replications. That is, you should have the average (across replications) of the average (within replications) waiting time for every combination of our design variables.**

**Task 2.7: Fit a (multiple) linear regression using the average (of the average) waiting time as dependent variable and our two design variables as independent variables. Note that the number of machines is a categorical variable and the reduction factor is a numerical variable. You can use any package for this, check Hint 1 for a suggestion.** 

**Task 2.8: Interpret the results of the (multiple) linear regression and answer the questions.**

-----

**Hint 1:** An easy way of fitting linear regression models in python is to use the `ols` function of the `statsmodels.formula.api` module. Once you have your fitted model, you only have to use the `.summary()` method on the model to get an overview of the regression results. Check the [reference](https://www.statsmodels.org/dev/example_formulas.html) for more details. Check as well the use of the `C()` operator for treating categorical variables.

In [22]:
class EventLogger_Task2:
    def __init__(self):
        self.logs = []
        self.replication = None
        self.seed = None
        self.system_id = None #used to identify the system in the logs
    
    def set_system_id(self, system_id):
        self.system_id = system_id

    def set_replication_info(self, replication, seed):
        self.replication = replication
        self.seed = seed

    def set_params(self, n_machines, reduction_factor):
        self.n_machines = n_machines
        self.reduction_factor = reduction_factor

    def log_car_arrival(self, entity, time, interarrival_time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'n_machines': self.n_machines, 'reduction_factor': self.reduction_factor, #system parameters
                          'event_time': time, 'event_name': 'car_arrival', 'event_key': entity, #simulation info
                          'interarrival_time': interarrival_time
                          })
    
    def log_car_wash_request(self, entity, time, machine_id, processing_time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'n_machines': self.n_machines, 'reduction_factor': self.reduction_factor, #system parameters
                          'event_time': time, 'event_name': 'car_wash_request', 'event_key': entity, #simulation info
                          'machine_id': machine_id, 'processing_time': processing_time
                          })

    def log_car_departure(self, entity, time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'n_machines': self.n_machines, 'reduction_factor': self.reduction_factor, #system parameters
                          'event_time': time, 'event_name': 'car_departure', 'event_key': entity #simulation info
                          })

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

In [23]:
def car_generator_crn(env, carwash, logger, stream_iat, stream_pt):
    '''
    stream_iat: inter-arrival time stream
    stream_pt: processing time stream
    '''
    car_count = 0
    
    # Create cars while the simulation is running
    while True:
        iat = stream_iat[car_count] #inter-arrival time for car "i"
        pt = stream_pt[car_count] #processing time time for car "i"

        yield env.timeout(iat) #common random numbers

        Car_S1(env, f'Car {car_count}', carwash, logger, iat, pt)
        car_count += 1

In [24]:
# parameters
SIM_TIME = 8*60    # Simulation time in minutes
N_REPLICATIONS = 10 # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger_t2 = EventLogger_Task2()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)
    stream_iat = np.random.exponential(1/0.5, 10000)
    stream_pt = np.random.exponential(1/0.3, 10000)

    #set replication id and random seed
    logger_t2.set_replication_info(i, seed)

    for n_machines in [1, 2, 3, 4]:
        for reduction_factor in list(np.arange(0, 0.8, 0.1)):

            #adjust the processing time stream
            stream_pt_adj = stream_pt * (1 - reduction_factor)

            #set parameters for logger
            logger_t2.set_params(n_machines, reduction_factor)

            # Create an environment and start the setup process
            env = simpy.Environment()

            #define resources
            carwash_s1 = Carwash_S1(env, logger_t2, n_machines)

            #define processes
            env.process(car_generator_crn(env, carwash_s1, logger_t2, stream_iat, stream_pt_adj))

            # Execute
            env.run(until=SIM_TIME)

    
print('... Done')

Running Simulation...
Running Replication 0 with seed: 93 ...
Running Replication 1 with seed: 95 ...
Running Replication 2 with seed: 53 ...
Running Replication 3 with seed: 19 ...
Running Replication 4 with seed: 64 ...
Running Replication 5 with seed: 52 ...
Running Replication 6 with seed: 97 ...
Running Replication 7 with seed: 47 ...
Running Replication 8 with seed: 56 ...
Running Replication 9 with seed: 32 ...
... Done


In [25]:
#Leave this code unchanged
events_df_t2 = logger_t2.get_logs_df()

events_df_t2

,system_id,replication_id,seed,n_machines,reduction_factor,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,None,0,93,1,0.0,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
1,None,0,93,1,0.0,1.864608,car_wash_request,Car 0,NaN,unique_machine,12.839254
2,None,0,93,1,0.0,4.007643,car_arrival,Car 1,2.143035,NaN,NaN
3,None,0,93,1,0.0,4.741789,car_arrival,Car 2,0.734146,NaN,NaN
4,None,0,93,1,0.0,11.282311,car_arrival,Car 3,6.540522,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
228006,None,9,32,4,0.7,479.568522,car_departure,Car 247,NaN,NaN,NaN
228007,None,9,32,4,0.7,479.678252,car_departure,Car 246,NaN,NaN,NaN
228008,None,9,32,4,0.7,479.817997,car_arrival,Car 248,0.597250,NaN,NaN
228009,None,9,32,4,0.7,479.817997,car_wash_request,Car 248,NaN,unique_machine,0.141654


In [26]:
#Leave this code unchanged (check that the interarrival times are the same for the same car for every combination of the design variables)
boolean_mask = (events_df_t2['replication_id'] == 0) & (events_df_t2['event_key'].isin(['Car 0']) ) & (events_df_t2['event_name'] == 'car_arrival')

events_df_t2[boolean_mask].sort_values(by=['event_key', 'event_name', 'n_machines', 'reduction_factor']).head(10)

,system_id,replication_id,seed,n_machines,reduction_factor,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,None,0,93,1,0.0,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
554,None,0,93,1,0.1,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
1130,None,0,93,1,0.2,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
1740,None,0,93,1,0.3,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
2380,None,0,93,1,0.4,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
3090,None,0,93,1,0.5,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
3843,None,0,93,1,0.6,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
4596,None,0,93,1,0.7,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
5349,None,0,93,2,0.0,1.864608,car_arrival,Car 0,1.864608,NaN,NaN
6098,None,0,93,2,0.1,1.864608,car_arrival,Car 0,1.864608,NaN,NaN


In [27]:
#Leave this code unchanged (check that the processing times are the adjusted for the same car for every combination of the design variables)

boolean_mask = (events_df_t2['replication_id'] == 0) & (events_df_t2['event_key'].isin(['Car 0']) ) & (events_df_t2['event_name'] == 'car_wash_request')

events_df_t2[boolean_mask].sort_values(by=['event_key', 'event_name', 'n_machines', 'reduction_factor']).head(10)

,system_id,replication_id,seed,n_machines,reduction_factor,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
1,None,0,93,1,0.0,1.864608,car_wash_request,Car 0,NaN,unique_machine,12.839254
555,None,0,93,1,0.1,1.864608,car_wash_request,Car 0,NaN,unique_machine,11.555329
1131,None,0,93,1,0.2,1.864608,car_wash_request,Car 0,NaN,unique_machine,10.271403
1741,None,0,93,1,0.3,1.864608,car_wash_request,Car 0,NaN,unique_machine,8.987478
2381,None,0,93,1,0.4,1.864608,car_wash_request,Car 0,NaN,unique_machine,7.703553
3091,None,0,93,1,0.5,1.864608,car_wash_request,Car 0,NaN,unique_machine,6.419627
3844,None,0,93,1,0.6,1.864608,car_wash_request,Car 0,NaN,unique_machine,5.135702
4597,None,0,93,1,0.7,1.864608,car_wash_request,Car 0,NaN,unique_machine,3.851776
5350,None,0,93,2,0.0,1.864608,car_wash_request,Car 0,NaN,unique_machine,12.839254
6099,None,0,93,2,0.1,1.864608,car_wash_request,Car 0,NaN,unique_machine,11.555329


In [28]:
#Compute the waiting time for each car in each replication and each combination of the design variables (you may want to use pd.pivot_table for this)

results_df_t2 = pd.pivot_table(events_df_t2, 
                            values='event_time', 
                            index=['replication_id', 'seed', 'n_machines', 'reduction_factor', 'event_key'], 
                            columns=['event_name'], aggfunc="sum")\
                .reset_index(drop=False)

#compute waiting time
results_df_t2['waiting_time'] = results_df_t2['car_wash_request'] - results_df_t2['car_arrival']

print(results_df_t2.columns)

results_df_t2

Index(['replication_id', 'seed', 'n_machines', 'reduction_factor', 'event_key',
       'car_arrival', 'car_departure', 'car_wash_request', 'waiting_time'],
      dtype='object', name='event_name')


event_name,replication_id,seed,n_machines,reduction_factor,event_key,car_arrival,car_departure,car_wash_request,waiting_time
0,0,93,1,0.0,Car 0,1.864608,14.703862,1.864608,0.000000
1,0,93,1,0.0,Car 1,4.007643,20.626759,14.703862,10.696219
2,0,93,1,0.0,Car 10,37.334984,60.684772,58.867303,21.532319
3,0,93,1,0.0,Car 100,197.258045,316.980062,316.174994,118.916950
4,0,93,1,0.0,Car 101,199.067657,317.061426,316.980062,117.912406
...,...,...,...,...,...,...,...,...,...
78363,9,32,4,0.7,Car 95,175.083602,175.945072,175.083602,0.000000
78364,9,32,4,0.7,Car 96,178.352318,178.916519,178.352318,0.000000
78365,9,32,4,0.7,Car 97,182.587011,182.926281,182.587011,0.000000
78366,9,32,4,0.7,Car 98,184.355090,185.396853,184.355090,0.000000


In [29]:
results_df_t2\
    .loc[results_df_t2['replication_id'] == 0]\
    .sort_values(by='car_arrival', ascending=True)\
    .head(10)

event_name,replication_id,seed,n_machines,reduction_factor,event_key,car_arrival,car_departure,car_wash_request,waiting_time
0,0,93,1,0.0,Car 0,1.864608,14.703862,1.864608,0.0
2761,0,93,2,0.3,Car 0,1.864608,10.852086,1.864608,0.0
7530,0,93,4,0.6,Car 0,1.864608,7.000310,1.864608,0.0
5271,0,93,3,0.5,Car 0,1.864608,8.284235,1.864608,0.0
6526,0,93,4,0.2,Car 0,1.864608,12.136012,1.864608,0.0
1004,0,93,1,0.4,Car 0,1.864608,9.568161,1.864608,0.0
1757,0,93,1,0.7,Car 0,1.864608,5.716384,1.864608,0.0
3765,0,93,2,0.7,Car 0,1.864608,5.716384,1.864608,0.0
5020,0,93,3,0.4,Car 0,1.864608,9.568161,1.864608,0.0
5522,0,93,3,0.6,Car 0,1.864608,7.000310,1.864608,0.0


In [30]:
#Compute the average waiting time for each replication and each combination of the design variables (average waiting time within replications, for each combination of the design variables)

within_replication_stats_df_t2 = results_df_t2\
    .groupby(['replication_id', 'n_machines', 'reduction_factor'])\
    .agg({'waiting_time': ['mean']})\
    .reset_index(drop=False)

print(within_replication_stats_df_t2.shape)
print(within_replication_stats_df_t2.columns)
within_replication_stats_df_t2.head()

(320, 4)
MultiIndex([(  'replication_id',     ''),
            (      'n_machines',     ''),
            ('reduction_factor',     ''),
            (    'waiting_time', 'mean')],
           names=['event_name', None])


event_name,replication_id,n_machines,reduction_factor,waiting_time
,,,,mean
0,0,1,0.0,90.516917
1,0,1,0.1,71.599195
2,0,1,0.2,53.402806
3,0,1,0.3,36.916341
4,0,1,0.4,17.199287


In [31]:
# Aggregate accross replications. 
# Compute the average (of the average) waiting time for each combination of the design variables (average waiting time across replications, for each combination of the design variables)

accross_replication_stats_df_t2 = within_replication_stats_df_t2\
    .groupby(['n_machines', 'reduction_factor'])\
    .agg({('waiting_time', 'mean'): ['mean']})\
    .reset_index(drop=False)

print(accross_replication_stats_df_t2.shape)
print(accross_replication_stats_df_t2.columns)
accross_replication_stats_df_t2.head()

(32, 3)
MultiIndex([(      'n_machines',     '',     ''),
            ('reduction_factor',     '',     ''),
            (    'waiting_time', 'mean', 'mean')],
           names=['event_name', None, None])


event_name,n_machines,reduction_factor,waiting_time
,,,mean
,,,mean
0,1,0.0,90.450288
1,1,0.1,74.243887
2,1,0.2,57.390495
3,1,0.3,40.532041
4,1,0.4,20.000555


In [32]:
#change the column names for convenience
accross_replication_stats_df_t2.columns = ['n_machines', 'reduction_factor', 'mean_waiting_time']

print(accross_replication_stats_df_t2.shape)
print(accross_replication_stats_df_t2.columns)
accross_replication_stats_df_t2.head()

(32, 3)
Index(['n_machines', 'reduction_factor', 'mean_waiting_time'], dtype='object')


,n_machines,reduction_factor,mean_waiting_time
0,1,0.0,90.450288
1,1,0.1,74.243887
2,1,0.2,57.390495
3,1,0.3,40.532041
4,1,0.4,20.000555


In [33]:
accross_replication_stats_df_t2.tail()

,n_machines,reduction_factor,mean_waiting_time
27,4,0.3,0.034420
28,4,0.4,0.016432
29,4,0.5,0.006159
30,4,0.6,0.001874
31,4,0.7,0.000202


In [47]:
#Fit a linear regression model to the data and interpret the results

model = smf.ols(formula='mean_waiting_time ~ C(n_machines) + reduction_factor', data=accross_replication_stats_df_t2).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      mean_waiting_time   R-squared:                       0.624
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     11.18
Date:                Tue, 11 Jun 2024   Prob (F-statistic):           1.76e-05
Time:                        09:47:21   Log-Likelihood:                -129.33
No. Observations:                  32   AIC:                             268.7
Df Residuals:                      27   BIC:                             276.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             49.5745      6

Answer the following questions:

- What can you say about the "goodnes-of-fit" of the model?
- Assuming you have only 1 machine, what would be better for reducing the average waiting time? (assume both options have the same cost)
    1. Buying one more machine 
    1. Upgrading the machine and reducing the processing time by 50%?. 
- How is the effect of buying more machines? Is this effect linear with the number of machines??

**Answer in this markdown chunk:**

- *Your answer here* (**Answer:** The model explains 62% of the variability of the dependent variable (average waiting time). The model is (overall) significant due the p-value of the F-test)
- *Your answer here* (**Answer:** The fitted model (rounded to the decimal) is:
    
     $avg\_waiting\_time = 49.6 + (-34.8) * n\_machine_2 + (-36.5) * n\_machine_3 + (-36.7) * n\_machine_4 + (-36.7) * reduction\_factor$. 
     
     Let's evaluate the options:
    - Option 1: (Buying 1 more machine is equivalent to have 2 machines) $avg\_waiting\_time = 49.6 + (-34.8) * 1 + (-36.5) * 0 + (-36.7) * 0 + (-36.7) * 0 = 14.8$ 
    - Option 2: $avg\_waiting\_time = 49.6 + (-34.8) * 0 + (-36.5) * 0 + (-36.7) * 0 + (-36.7) * 0.5 = 31.3$

    Therefore, option 1 is better (buying one more machine) for reducing the average waiting time. 
)

- *Your answer here* (**Answer:** Using the mathematical formula, let's check the effect of buying more machines:
    -   +1 machine: $avg\_waiting\_time = 49.6 + (-34.8) * 1 + (-36.5) * 0 + (-36.7) * 0 + (-36.7) * 0 = 14.8$ (reduction of ~70% compared to 1 machine) 
    -   +2 machine: $avg\_waiting\_time = 49.6 + (-34.8) * 0 + (-36.5) * 1 + (-36.7) * 0 + (-36.7) * 0 = 13.1$ (reduction of ~73% compared to 1 machine)
    -   +3 machine: $avg\_waiting\_time = 49.6 + (-34.8) * 0 + (-36.5) * 0 + (-36.7) * 1 + (-36.7) * 0 = 12.9$ (reduction of ~74% compared to 1 machine)

    With this, is clearly that the effect of buying more machines **is not linear** but saturates while increasing the machines (similar to diminishing returns)
)